# Interpretable Machine Learning for Breast Cancer Diagnosis

In [ ]:
# Imports
from sklearn.datasets import load_breast_cancer
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

## 2. Dataset
We use the Breast Cancer Wisconsin Diagnostic dataset provided by scikit-learn.

In [ ]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

print(X.shape)
print(y.value_counts())

## 3. Methodology
The workflow includes splitting the dataset into training and test sets, training classification models, evaluating performance using ROC-AUC and confusion matrices, and interpreting model predictions.

In [ ]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=5000))
])

model.fit(X_train, y_train)

y_proba = model.predict_proba(X_test)[:, 1]
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

“Logistic Regression achieves a high ROC-AUC, indicating strong separability between malignant and benign samples while remaining interpretable for clinical reasoning.”

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)

rf_model.fit(X_train, y_train)

rf_proba = rf_model.predict_proba(X_test)[:, 1]
print("Random Forest ROC-AUC:", roc_auc_score(y_test, rf_proba))

### 3.1 Confusion Matrix
The confusion matrix is used to analyze classification errors, which is especially important in medical applications.

In [ ]:
## Confusion Matrix (Logistic Regression)

# Predictions from Logistic Regression
y_pred = model.predict(X_test)

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Display it
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues")
plt.title("Confusion Matrix - Logistic Regression")
plt.show()

## 4. Model Interpretability
Permutation importance is applied to identify the most influential features affecting model predictions.

In [ ]:
## Model Interpretability (Permutation Importance)

result = permutation_importance(
    model,
    X_test,
    y_test,
    n_repeats=10,
    random_state=42,
    n_jobs=-1
)

importances = pd.Series(
    result.importances_mean,
    index=X.columns
).sort_values(ascending=False)

importances.head(10)

In [ ]:

importances.head(10).plot(
    kind="barh",
    title="Top 10 Important Features (Permutation Importance)"
)
plt.gca().invert_yaxis()
plt.show()

## 5. Results, Limitations, and Conclusion

**Results:** The Logistic Regression model achieved strong ROC-AUC performance, indicating effective separation between malignant and benign tumors.

**Limitations:** This project uses a single curated dataset and classical machine learning models. Future work could include validation on external clinical datasets.

**Conclusion:** This project highlights the importance of interpretable machine learning techniques in healthcare-focused applications.